# Tensorflow intro

This tutorial shows the basic usage of tensorflow to train neural networks

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Simple NN for classification

## Data processing
Read data and convert them to numerical inputs

In [3]:
X, y = make_classification(n_samples=20000, n_features=8, n_informative=5, 
                           n_redundant=0, n_classes=2, random_state=1)

In [4]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
print('train size: {}, target_ratio: {:.3f}'.format(X_train.shape, np.mean(y_train)))
print('test size: {}, target_ratio: {:.3f}'.format(X_test.shape, np.mean(y_test)))
print('dev size: {}, target_ratio: {:.3f}'.format(X_dev.shape, np.mean(y_dev)))

train size: (12800, 8), target_ratio: 0.502
test size: (4000, 8), target_ratio: 0.501
dev size: (3200, 8), target_ratio: 0.502


## Building a simple model with tf.keras

Very useful documentations with many examples and detailed explanation of everything you might need:
 - https://www.tensorflow.org/api_docs/python/tf/keras/
 - https://keras.io/api/

Contain everything about:
  - Model building: Activations, Losses, Optimizers, Regularization
  - Data processing
  - Pretrained models and datasets
  - Automatic differentiation
  - ...

  

### Model speficication

three APIs for building the model
   - sequential - easy to code, but less flexible - we will use it sometimes
   - functional - flexible and still easy to code - we will use it the most
   - model subclassing - rather complicated and not very much used - we will skip it

#### Sequential API

(https://www.tensorflow.org/guide/keras/sequential_model)

Easy to code but not appropriate when:

- Your model has multiple inputs or multiple outputs
- Any of your layers has multiple inputs or multiple outputs
- You need to do layer sharing
- You want non-linear topology (e.g. a residual connection, a multi-branch model)

In [7]:
# Specification A)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer([X_train.shape[1],]), # Create input layer with 'input data' neurons
    tf.keras.layers.Dense(10, activation="relu"), # Create hidden layer with 10 neurons and ReLU activation
    tf.keras.layers.Dense(1, activation="sigmoid"), # Create output layer with one neuron and sigmoid activation
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                90        
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Specification B)

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(10, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                90        
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


#### Functional API

(https://www.tensorflow.org/guide/keras/functional)

The Keras functional API is a way to create models that are more flexible than the tf.keras.Sequential API. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.

The main idea is that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build graphs of layers.



In [9]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

hidden = tf.keras.layers.Dense(10)(inputs)
hidden = tf.keras.activations.relu(hidden)
hidden = tf.keras.layers.Dense(1)(hidden)
outputs = tf.keras.activations.sigmoid(hidden)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='Model')

In [10]:
model.summary()

Model: "Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_4 (Dense)             (None, 10)                90        
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 10)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
 tf.math.sigmoid (TFOpLambda  (None, 1)                0         
 )                                                               
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_______________________________________________________________

### Model compilation and training

In [11]:
# compile the model with selected optimizer, loss and metrics
model.compile(
        optimizer=tf.optimizers.Adam(), # Several other possibilities for optimizers 
        loss=tf.losses.BinaryCrossentropy(), # Select the proper loss for the task
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()], # Select the proper metrics for the task
)

In [12]:
print('\n>>> Bias of the last layers:')
print(model.layers[3].weights[1].numpy())

print('\n>>> Kernel of the last layers:')
print(model.layers[3].weights[0].numpy())

print('\n>>> Bias of the first layers:')
print(model.layers[1].weights[1].numpy())

print('\n>>> Kernel of the first layers:')
print(model.layers[1].weights[0].numpy())


>>> Bias of the last layers:
[0.]

>>> Kernel of the last layers:
[[-0.24726865]
 [ 0.68438464]
 [ 0.14847296]
 [-0.11615735]
 [-0.13499832]
 [ 0.06303412]
 [-0.12831736]
 [-0.6355281 ]
 [-0.73033303]
 [-0.03700101]]

>>> Bias of the first layers:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

>>> Kernel of the first layers:
[[ 0.27434552 -0.08809695 -0.34831077  0.05812716 -0.26086658  0.5021044
   0.02122277 -0.12708426  0.0413698   0.38695037]
 [-0.28205135 -0.0381906  -0.54502565 -0.518739    0.57005167 -0.2850911
  -0.28100067  0.5341382   0.12164605  0.29330844]
 [-0.11227959 -0.00172889  0.4041726  -0.1026282   0.18625504  0.16321474
   0.05342579 -0.02886128 -0.09993783 -0.2026555 ]
 [ 0.0565933   0.14265877  0.48244488 -0.14600396  0.26096338  0.13438952
   0.48842633  0.32555115 -0.00207978  0.01282179]
 [ 0.07784826 -0.03354377 -0.31859687 -0.06408441  0.2024464  -0.15602511
  -0.38213772  0.4802389   0.04864037 -0.16675308]
 [ 0.4762888   0.5667201  -0.12803513 -0.37903354 -0.34179136 -

In [13]:
# train the model with default setting
model.fit(X_train, y_train, batch_size=64, epochs=100)

Epoch 1/100
200/200 [==============================] - 1s 1ms/step - loss: 0.6496 - auc: 0.6642 - binary_accuracy: 0.6167
Epoch 2/100
200/200 [==============================] - 0s 1ms/step - loss: 0.5052 - auc: 0.8566 - binary_accuracy: 0.7827
Epoch 3/100
200/200 [==============================] - 0s 1ms/step - loss: 0.4307 - auc: 0.8942 - binary_accuracy: 0.8109
Epoch 4/100
200/200 [==============================] - 0s 1ms/step - loss: 0.3784 - auc: 0.9172 - binary_accuracy: 0.8402
Epoch 5/100
200/200 [==============================] - 0s 1ms/step - loss: 0.3442 - auc: 0.9299 - binary_accuracy: 0.8583
Epoch 6/100
200/200 [==============================] - 0s 1ms/step - loss: 0.3246 - auc: 0.9369 - binary_accuracy: 0.8675
Epoch 7/100
200/200 [==============================] - 0s 1ms/step - loss: 0.3120 - auc: 0.9412 - binary_accuracy: 0.8727
Epoch 8/100
200/200 [==============================] - 0s 1ms/step - loss: 0.3031 - auc: 0.9440 - binary_accuracy: 0.8797
Epoch 9/100
200/200 [===

200/200 [==============================] - 0s 1ms/step - loss: 0.2522 - auc: 0.9607 - binary_accuracy: 0.9082
Epoch 68/100
200/200 [==============================] - 0s 1ms/step - loss: 0.2517 - auc: 0.9609 - binary_accuracy: 0.9077
Epoch 69/100
200/200 [==============================] - 0s 1ms/step - loss: 0.2514 - auc: 0.9610 - binary_accuracy: 0.9087
Epoch 70/100
200/200 [==============================] - 0s 1ms/step - loss: 0.2513 - auc: 0.9610 - binary_accuracy: 0.9093
Epoch 71/100
200/200 [==============================] - 0s 1ms/step - loss: 0.2508 - auc: 0.9610 - binary_accuracy: 0.9096
Epoch 72/100
200/200 [==============================] - 0s 1ms/step - loss: 0.2506 - auc: 0.9614 - binary_accuracy: 0.9091
Epoch 73/100
200/200 [==============================] - 0s 1ms/step - loss: 0.2503 - auc: 0.9612 - binary_accuracy: 0.9085
Epoch 74/100
200/200 [==============================] - 0s 1ms/step - loss: 0.2500 - auc: 0.9614 - binary_accuracy: 0.9102
Epoch 75/100
200/200 [=======

In [14]:
# Evaluate the model and predict for the test data
model.evaluate(X_test, y_test)
test_pred = model.predict(X_test)

125/125 [==============================] - 0s 1ms/step


In [15]:
for pred, true in zip(test_pred, y_test[0:10]):
    print('{} - {:.2f}'.format(true, pred[0]))

1 - 0.56
0 - 0.27
0 - 0.52
1 - 0.97
0 - 0.00
0 - 0.02
1 - 0.89
0 - 0.01
1 - 0.95
1 - 1.00


### Add early stopping and regularization

In [16]:
# Input layer
inputs = tf.keras.Input(shape=(X_train.shape[1]))

# Hidden layer with regularization and ReLU
hidden = tf.keras.layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2(0.001))(inputs)
hidden = tf.keras.activations.relu(hidden)

# Output layer with regularization and sigmoid
outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.001))(hidden)
outputs = tf.keras.activations.sigmoid(outputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

model.compile(
        optimizer=tf.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()],
)

model.summary()

Model: "RegularizedModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_6 (Dense)             (None, 10)                90        
                                                                 
 tf.nn.relu_1 (TFOpLambda)   (None, 10)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
 tf.math.sigmoid_1 (TFOpLamb  (None, 1)                0         
 da)                                                             
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
____________________________________________________

In [17]:
epochs = 200

early_call = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

model.fit(X_train,
          y_train,
          epochs=epochs,
          validation_data=(X_dev, y_dev),
          callbacks=[early_call])

Epoch 1/200
400/400 [==============================] - 2s 2ms/step - loss: 0.6190 - auc_1: 0.7656 - binary_accuracy: 0.6721 - val_loss: 0.4471 - val_auc_1: 0.9062 - val_binary_accuracy: 0.7975
Epoch 2/200
400/400 [==============================] - 1s 1ms/step - loss: 0.4075 - auc_1: 0.9132 - binary_accuracy: 0.8209 - val_loss: 0.3698 - val_auc_1: 0.9300 - val_binary_accuracy: 0.8587
Epoch 3/200
400/400 [==============================] - 1s 1ms/step - loss: 0.3586 - auc_1: 0.9330 - binary_accuracy: 0.8584 - val_loss: 0.3350 - val_auc_1: 0.9428 - val_binary_accuracy: 0.8769
Epoch 4/200
400/400 [==============================] - 0s 1ms/step - loss: 0.3275 - auc_1: 0.9449 - binary_accuracy: 0.8777 - val_loss: 0.3104 - val_auc_1: 0.9506 - val_binary_accuracy: 0.8909
Epoch 5/200
400/400 [==============================] - 1s 1ms/step - loss: 0.3076 - auc_1: 0.9518 - binary_accuracy: 0.8894 - val_loss: 0.2960 - val_auc_1: 0.9560 - val_binary_accuracy: 0.8981
Epoch 6/200
400/400 [==============

400/400 [==============================] - 1s 2ms/step - loss: 0.2372 - auc_1: 0.9775 - binary_accuracy: 0.9301 - val_loss: 0.2355 - val_auc_1: 0.9777 - val_binary_accuracy: 0.9309
Epoch 44/200
400/400 [==============================] - 1s 2ms/step - loss: 0.2365 - auc_1: 0.9776 - binary_accuracy: 0.9305 - val_loss: 0.2352 - val_auc_1: 0.9781 - val_binary_accuracy: 0.9322
Epoch 45/200
400/400 [==============================] - 1s 2ms/step - loss: 0.2360 - auc_1: 0.9780 - binary_accuracy: 0.9307 - val_loss: 0.2346 - val_auc_1: 0.9780 - val_binary_accuracy: 0.9341
Epoch 46/200
400/400 [==============================] - 1s 2ms/step - loss: 0.2356 - auc_1: 0.9780 - binary_accuracy: 0.9315 - val_loss: 0.2348 - val_auc_1: 0.9782 - val_binary_accuracy: 0.9344
Epoch 47/200
400/400 [==============================] - 1s 2ms/step - loss: 0.2350 - auc_1: 0.9784 - binary_accuracy: 0.9321 - val_loss: 0.2336 - val_auc_1: 0.9783 - val_binary_accuracy: 0.9322
Epoch 48/200
400/400 [=====================

400/400 [==============================] - 1s 2ms/step - loss: 0.2284 - auc_1: 0.9812 - binary_accuracy: 0.9390 - val_loss: 0.2292 - val_auc_1: 0.9811 - val_binary_accuracy: 0.9375
Epoch 86/200
400/400 [==============================] - 1s 2ms/step - loss: 0.2283 - auc_1: 0.9815 - binary_accuracy: 0.9391 - val_loss: 0.2287 - val_auc_1: 0.9812 - val_binary_accuracy: 0.9381
Epoch 87/200
400/400 [==============================] - 1s 2ms/step - loss: 0.2284 - auc_1: 0.9814 - binary_accuracy: 0.9384 - val_loss: 0.2296 - val_auc_1: 0.9810 - val_binary_accuracy: 0.9388
Epoch 88/200
400/400 [==============================] - 1s 2ms/step - loss: 0.2279 - auc_1: 0.9815 - binary_accuracy: 0.9391 - val_loss: 0.2299 - val_auc_1: 0.9813 - val_binary_accuracy: 0.9406
Epoch 89/200
400/400 [==============================] - 1s 2ms/step - loss: 0.2281 - auc_1: 0.9816 - binary_accuracy: 0.9397 - val_loss: 0.2287 - val_auc_1: 0.9812 - val_binary_accuracy: 0.9378
Epoch 90/200
400/400 [=====================

### Gridsearch and Tensorboard
Run gridsearch over hidden layer size, L2 regularization, activation, check the outputs in Tensorboard

I recommend not to run Tensorboard from Jupyter notebook but from terminal directly

use "tensorboard --logdir logs" in command line

In [18]:
# hidden_sizes = [2, 5, 10, 20, 50]
# l2_regs = [0.01, 0.001, 0.0001]
# activations = ['relu', 'tanh']
# ensorboard --logdir="C:\Users\Karla\Documents\Data Science 2\Practicals\05_neural_networks\logs\binary_classification_test"

hidden_sizes = [2, 5]
l2_regs = [0.01]
activations = ['relu', 'tanh']

epochs = 200
batch_size = 64

early_call = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', mode='max', patience=10, restore_best_weights=True)

for activation in activations:
    for l2_reg in l2_regs:
        for hidden_size in hidden_sizes:
            if activation == 'relu':
                activate = tf.keras.activations.relu
            elif activation == 'tanh':
                activate = tf.keras.activations.tanh

            # Create Tensorboard Callback
            param_string = 'act-{},l2-{},hs-{}'.format(activation, l2_reg, hidden_size)
            log_dir = 'logs/binary_classification_test/' + param_string
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

            # Input layer
            inputs = tf.keras.Input(shape=(X_train.shape[1]))

            # Hidden layer with regularization and ReLU
            hidden = tf.keras.layers.Dense(hidden_size, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(inputs)
            hidden = activate(hidden)

            # Output layer with regularization and sigmoid
            outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(hidden)
            outputs = tf.keras.activations.sigmoid(outputs)

            model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

            model.compile(
                    optimizer=tf.optimizers.Adam(),
                    loss=tf.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.AUC(name='AUC'), tf.keras.metrics.BinaryAccuracy()],
            )

            # Train the model
            model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                      validation_data=(X_dev, y_dev),
                      callbacks=[early_call, tensorboard_callback])

Epoch 1/200
200/200 [==============================] - 2s 4ms/step - loss: 0.8446 - AUC: 0.3847 - binary_accuracy: 0.4023 - val_loss: 0.7682 - val_AUC: 0.4060 - val_binary_accuracy: 0.4153
Epoch 2/200
200/200 [==============================] - 0s 2ms/step - loss: 0.7407 - AUC: 0.4627 - binary_accuracy: 0.4601 - val_loss: 0.7094 - val_AUC: 0.5541 - val_binary_accuracy: 0.5269
Epoch 3/200
200/200 [==============================] - 1s 3ms/step - loss: 0.6771 - AUC: 0.6743 - binary_accuracy: 0.6305 - val_loss: 0.6373 - val_AUC: 0.7702 - val_binary_accuracy: 0.7019
Epoch 4/200
200/200 [==============================] - 0s 2ms/step - loss: 0.6171 - AUC: 0.7885 - binary_accuracy: 0.7215 - val_loss: 0.5911 - val_AUC: 0.8178 - val_binary_accuracy: 0.7597
Epoch 5/200
200/200 [==============================] - 1s 3ms/step - loss: 0.5829 - AUC: 0.8219 - binary_accuracy: 0.7644 - val_loss: 0.5624 - val_AUC: 0.8438 - val_binary_accuracy: 0.7819
Epoch 6/200
200/200 [==============================] - 

200/200 [==============================] - 0s 2ms/step - loss: 0.4788 - AUC: 0.8878 - binary_accuracy: 0.8177 - val_loss: 0.4688 - val_AUC: 0.8943 - val_binary_accuracy: 0.8259
Epoch 45/200
200/200 [==============================] - 1s 3ms/step - loss: 0.4787 - AUC: 0.8879 - binary_accuracy: 0.8179 - val_loss: 0.4688 - val_AUC: 0.8942 - val_binary_accuracy: 0.8259
Epoch 46/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4786 - AUC: 0.8880 - binary_accuracy: 0.8175 - val_loss: 0.4688 - val_AUC: 0.8945 - val_binary_accuracy: 0.8269
Epoch 47/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4785 - AUC: 0.8882 - binary_accuracy: 0.8181 - val_loss: 0.4685 - val_AUC: 0.8945 - val_binary_accuracy: 0.8272
Epoch 48/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4783 - AUC: 0.8883 - binary_accuracy: 0.8183 - val_loss: 0.4690 - val_AUC: 0.8943 - val_binary_accuracy: 0.8247
Epoch 49/200
200/200 [==============================] - 1s 3ms/

200/200 [==============================] - 1s 3ms/step - loss: 0.4765 - AUC: 0.8900 - binary_accuracy: 0.8196 - val_loss: 0.4673 - val_AUC: 0.8961 - val_binary_accuracy: 0.8259
Epoch 88/200
200/200 [==============================] - 1s 3ms/step - loss: 0.4765 - AUC: 0.8902 - binary_accuracy: 0.8192 - val_loss: 0.4674 - val_AUC: 0.8957 - val_binary_accuracy: 0.8241
Epoch 89/200
200/200 [==============================] - 1s 3ms/step - loss: 0.4764 - AUC: 0.8900 - binary_accuracy: 0.8198 - val_loss: 0.4677 - val_AUC: 0.8958 - val_binary_accuracy: 0.8241
Epoch 90/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4764 - AUC: 0.8899 - binary_accuracy: 0.8188 - val_loss: 0.4672 - val_AUC: 0.8961 - val_binary_accuracy: 0.8256
Epoch 91/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4764 - AUC: 0.8901 - binary_accuracy: 0.8188 - val_loss: 0.4673 - val_AUC: 0.8959 - val_binary_accuracy: 0.8256
Epoch 92/200
200/200 [==============================] - 1s 3ms/

200/200 [==============================] - 0s 2ms/step - loss: 0.4335 - AUC: 0.9310 - binary_accuracy: 0.8225 - val_loss: 0.4274 - val_AUC: 0.9358 - val_binary_accuracy: 0.8309
Epoch 12/200
200/200 [==============================] - 1s 3ms/step - loss: 0.4304 - AUC: 0.9335 - binary_accuracy: 0.8329 - val_loss: 0.4247 - val_AUC: 0.9381 - val_binary_accuracy: 0.8553
Epoch 13/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4255 - AUC: 0.9369 - binary_accuracy: 0.8637 - val_loss: 0.4201 - val_AUC: 0.9414 - val_binary_accuracy: 0.8766
Epoch 14/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4213 - AUC: 0.9402 - binary_accuracy: 0.8740 - val_loss: 0.4168 - val_AUC: 0.9439 - val_binary_accuracy: 0.8813
Epoch 15/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4170 - AUC: 0.9437 - binary_accuracy: 0.8790 - val_loss: 0.4137 - val_AUC: 0.9456 - val_binary_accuracy: 0.8772
Epoch 16/200
200/200 [==============================] - 0s 2ms/

200/200 [==============================] - 1s 3ms/step - loss: 0.3994 - AUC: 0.9552 - binary_accuracy: 0.8893 - val_loss: 0.3992 - val_AUC: 0.9549 - val_binary_accuracy: 0.8903
Epoch 55/200
200/200 [==============================] - 0s 2ms/step - loss: 0.3994 - AUC: 0.9551 - binary_accuracy: 0.8897 - val_loss: 0.3991 - val_AUC: 0.9551 - val_binary_accuracy: 0.8891
Epoch 56/200
200/200 [==============================] - 0s 2ms/step - loss: 0.3993 - AUC: 0.9553 - binary_accuracy: 0.8904 - val_loss: 0.3988 - val_AUC: 0.9549 - val_binary_accuracy: 0.8891
Epoch 57/200
200/200 [==============================] - 1s 3ms/step - loss: 0.3993 - AUC: 0.9550 - binary_accuracy: 0.8893 - val_loss: 0.3994 - val_AUC: 0.9549 - val_binary_accuracy: 0.8881
Epoch 58/200
200/200 [==============================] - 0s 2ms/step - loss: 0.3994 - AUC: 0.9551 - binary_accuracy: 0.8892 - val_loss: 0.3990 - val_AUC: 0.9550 - val_binary_accuracy: 0.8903
Epoch 59/200
200/200 [==============================] - 1s 2ms/

200/200 [==============================] - 0s 2ms/step - loss: 0.4894 - AUC: 0.8796 - binary_accuracy: 0.7923 - val_loss: 0.4797 - val_AUC: 0.8890 - val_binary_accuracy: 0.8106
Epoch 24/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4893 - AUC: 0.8800 - binary_accuracy: 0.7935 - val_loss: 0.4794 - val_AUC: 0.8889 - val_binary_accuracy: 0.8106
Epoch 25/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4893 - AUC: 0.8800 - binary_accuracy: 0.7929 - val_loss: 0.4795 - val_AUC: 0.8889 - val_binary_accuracy: 0.8094
Epoch 26/200
200/200 [==============================] - 1s 3ms/step - loss: 0.4892 - AUC: 0.8802 - binary_accuracy: 0.7936 - val_loss: 0.4797 - val_AUC: 0.8888 - val_binary_accuracy: 0.8106
Epoch 27/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4892 - AUC: 0.8801 - binary_accuracy: 0.7935 - val_loss: 0.4796 - val_AUC: 0.8892 - val_binary_accuracy: 0.8112
Epoch 28/200
200/200 [==============================] - 1s 3ms/

200/200 [==============================] - 1s 3ms/step - loss: 0.4550 - AUC: 0.9256 - binary_accuracy: 0.8448 - val_loss: 0.4471 - val_AUC: 0.9314 - val_binary_accuracy: 0.8631
Epoch 27/200
200/200 [==============================] - 1s 3ms/step - loss: 0.4544 - AUC: 0.9257 - binary_accuracy: 0.8467 - val_loss: 0.4475 - val_AUC: 0.9330 - val_binary_accuracy: 0.8616
Epoch 28/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4540 - AUC: 0.9277 - binary_accuracy: 0.8493 - val_loss: 0.4461 - val_AUC: 0.9325 - val_binary_accuracy: 0.8622
Epoch 29/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4536 - AUC: 0.9270 - binary_accuracy: 0.8472 - val_loss: 0.4461 - val_AUC: 0.9334 - val_binary_accuracy: 0.8625
Epoch 30/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4531 - AUC: 0.9279 - binary_accuracy: 0.8494 - val_loss: 0.4458 - val_AUC: 0.9341 - val_binary_accuracy: 0.8609
Epoch 31/200
200/200 [==============================] - 0s 2ms/

200/200 [==============================] - 0s 2ms/step - loss: 0.4475 - AUC: 0.9375 - binary_accuracy: 0.8620 - val_loss: 0.4419 - val_AUC: 0.9413 - val_binary_accuracy: 0.8719
Epoch 70/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4473 - AUC: 0.9379 - binary_accuracy: 0.8623 - val_loss: 0.4418 - val_AUC: 0.9410 - val_binary_accuracy: 0.8728
Epoch 71/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4474 - AUC: 0.9381 - binary_accuracy: 0.8621 - val_loss: 0.4417 - val_AUC: 0.9409 - val_binary_accuracy: 0.8728
Epoch 72/200
200/200 [==============================] - 0s 2ms/step - loss: 0.4475 - AUC: 0.9376 - binary_accuracy: 0.8612 - val_loss: 0.4417 - val_AUC: 0.9414 - val_binary_accuracy: 0.8731
Epoch 73/200
200/200 [==============================] - 1s 3ms/step - loss: 0.4474 - AUC: 0.9381 - binary_accuracy: 0.8633 - val_loss: 0.4419 - val_AUC: 0.9413 - val_binary_accuracy: 0.8744
Epoch 74/200
200/200 [==============================] - 0s 2ms/